In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from cnmfsns.config import Config
from cnmfsns.containers import Integration
config = Config.from_toml("scripts/.toml")
data = Integration.from_config(config)

Reading /mnt/c/Users/verhe/MorrissyLab Dropbox/Proteomics/tmt_proteomics/cNMF/output/cptac_gbm_imputed_odg1_kl.h5mu
Reading /mnt/c/Users/verhe/MorrissyLab Dropbox/Proteomics/tmt_proteomics/cNMF/output/gbm_ffpe_proteogenomics_imputed_odg1_kl.h5mu
Reading /mnt/c/Users/verhe/MorrissyLab Dropbox/Proteomics/tmt_proteomics/cNMF/output/yonsei_imputed_odg1_kl.h5mu
Reading /mnt/c/Users/verhe/MorrissyLab Dropbox/Proteomics/tmt_proteomics/cNMF/output/diamandis_imputed_odg1_kl.h5mu


In [23]:
data.corr

CPTAC                                                    \
                       2                   3                             4    
                       1         2         1         2         3         1    
CPTAC     2  1   1.000000 -0.991545  0.521512 -0.995529  0.410736  0.370842   
             2  -0.991545  1.000000 -0.441538  0.996493 -0.486902 -0.447872   
          3  1   0.521512 -0.441538  1.000000 -0.514784 -0.563650 -0.573518   
             2  -0.995529  0.996493 -0.514784  1.000000 -0.412560 -0.373445   
             3   0.410736 -0.486902 -0.563650 -0.412560  1.000000  0.970462   
...                   ...       ...       ...       ...       ...       ...   
Diamandis 60 56  0.225460 -0.219235  0.114253 -0.218948  0.121286  0.090172   
             57 -0.730024  0.715035 -0.488237  0.723935 -0.260017 -0.194912   
             58 -0.224691  0.211956 -0.198204  0.217739 -0.026163 -0.058221   
             59 -0.505625  0.496077 -0.334535  0.501470 -0.183362 -0.155892   
             60  0.006025 -0.009965  0.005645 -0.010839  0.000803 -0.036029   

                                                         ... Diamandis  \
                                                     5   ...        60   
                       2         3         4         1   ...        51   
CPTAC     2  1  -0.991071  0.503739  0.051921  0.022899  ...  0.286183   
             2   0.992652 -0.429798 -0.041867 -0.133920  ... -0.297804   
          3  1  -0.507858  0.920544  0.107687 -0.578845  ... -0.034727   
             2   0.996094 -0.494420 -0.054305 -0.075128  ... -0.280424   
             3  -0.416152 -0.497979 -0.059066  0.633586  ...  0.349248   
...                   ...       ...       ...       ...  ...       ...   
Diamandis 60 56 -0.223738  0.085977  0.083029 -0.062012  ... -0.039181   
             57  0.730663 -0.417393 -0.180297  0.115445  ... -0.278301   
             58  0.201369 -0.253634  0.146437  0.050630  ... -0.105125   
             59  0.500125 -0.313853 -0.053396  0.062761  ... -0.212812   
             60 -0.023893 -0.056301  0.153032 -0.026545  ... -0.012976   

                                                                             \
                                                                              
                       52        53        54        55        56        57   
CPTAC     2  1   0.225754  0.272678  0.320106  0.199836  0.225460 -0.730024   
             2  -0.210456 -0.293146 -0.339297 -0.207336 -0.219235  0.715035   
          3  1   0.154838 -0.077815 -0.128472  0.081541  0.114253 -0.488237   
             2  -0.214118 -0.272340 -0.311585 -0.205295 -0.218948  0.723935   
             3   0.078258  0.380788  0.488767  0.127719  0.121286 -0.260017   
...                   ...       ...       ...       ...       ...       ...   
Diamandis 60 56  0.293252  0.088537  0.062290  0.005861  1.000000 -0.247759   
             57 -0.220943 -0.269845 -0.248419 -0.235203 -0.247759  1.000000   
             58 -0.057543 -0.120458 -0.150184 -0.025436 -0.016073  0.114257   
             59 -0.196361 -0.160555 -0.218122 -0.148655 -0.147725  0.477106   
             60 -0.059134  0.011282 -0.039544  0.004309 -0.009590 -0.109042   

                                               
                                               
                       58        59        60  
CPTAC     2  1  -0.224691 -0.505625  0.006025  
             2   0.211956  0.496077 -0.009965  
          3  1  -0.198204 -0.334535  0.005645  
             2   0.217739  0.501470 -0.010839  
             3  -0.026163 -0.183362  0.000803  
...                   ...       ...       ...  
Diamandis 60 56 -0.016073 -0.147725 -0.009590  
             57  0.114257  0.477106 -0.109042  
             58  1.000000  0.131321  0.207442  
             59  0.131321  1.000000 -0.032964  
             60  0.207442 -0.032964  1.000000  

[7316 rows x 7316 columns]

In [110]:
from anndata import AnnData, read_h5ad
import pandas as pd
import numpy as np
import logging
# def txt_to_h5ad(counts, tpm, metadata, output):
counts = None
tpm = "/mnt/c/Users/verhe/MorrissyLab Dropbox/Proteomics/tmt_proteomics/cNMF/data/imputed/cptac_gbm.tsv"
metadata = "/mnt/c/Users/verhe/MorrissyLab Dropbox/Proteomics/tmt_proteomics/sample_metadata/cnmfsns_metadata/cptac_gbm.metadata.txt"
output = "/mnt/c/Users/verhe/MorrissyLab Dropbox/Proteomics/tmt_proteomics/cNMF-SNS/integrations/cptac_gbm.h5ad"

adata = read_h5ad(output)
adata

AnnData object with n_obs × n_vars = 110 × 11293
    obs: 'patient_id', 'simple_category', 'detailed_category', 'purity_TSNet', 'multiomic', 'r_wang_cancer_cell_2017', 'd_methyl', 'd_methyl_mnp', 'is_gcimp', 'immune', 'mir'

In [116]:
# data.isnull().sum()/data.shape[0]

np.isnan(adata.X).sum(0)/adata.shape[0]

ValueError: operands could not be broadcast together with shapes (11293,) (2,) 